In [4]:
##outer boundary
import cv2
import numpy as np
import os
import json

path = input('Enter a file path: ')  # Path of the folder containing the masked images
dirs = os.listdir(path)

for item in dirs:
    if os.path.isfile(os.path.join(path, item)):
        filename, ext = os.path.splitext(item)
        if ext.lower() == '.png':  # Process only BMP files
            print(item)  # Name of the file
            image = cv2.imread(os.path.join(path, item))  # Load the image

            cv2.waitKey(0)
            x = image.shape  # Height and width of the image

            # Convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Apply adaptive thresholding
            thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

            # Find contours on the binary image
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            print("Number of Contours found =", len(contours))

            # Filter and draw contours around polygons
            filtered_contours = []
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 400:  # Adjust the threshold area value as needed
                    filtered_contours.append(contour)

            # Create a blank image to draw only the contours
            contour_image = np.zeros_like(image)

            cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 1)

            print("Number of Contours after filtering =", len(filtered_contours))

            json_path = os.path.join(path, filename + ".json")
            with open(json_path, "w+") as f:  # Same filename as image
                # Format of the JSON
                data = {
                    "version": "5.1.1",
                    "flags": {},
                    "shapes": [
                        {
                            "label": "product",
                            "points": [
                                (float(contour[j][0][0]),
                                 float(contour[j][0][1])) for j in range(len(contour))
                            ],
                            "group_id": None,
                            "shape_type": "polygon",
                            "flags": {}
                        } for contour in filtered_contours
                    ],
                    "imagePath": item,  # Image name
                    "imageData": None,
                    "imageHeight": x[0],
                    "imageWidth": x[1]
                }

                json.dump(data, f, indent=4)

            # Show the image with contours
            #cv2.imshow('Contours', contour_image)
            #cv2.waitKey(0)

cv2.destroyAllWindows()

Enter a file path: C:/Users/dhyan/Downloads/data/output_outerboundary/
oval.png
Number of Contours found = 401
Number of Contours after filtering = 7
pills.png
Number of Contours found = 37
Number of Contours after filtering = 4
